# Extensions to Linear Models - Lab

## Introduction

In this lab, you'll practice many concepts you have learned so far, from adding interactions and polynomials to your model to AIC and BIC!

## Summary

You will be able to:
- Build a linear regression model with interactions and polynomial features 
- Use AIC and BIC to select the best value for the regularization parameter 


## Let's get started!

Import all the necessary packages.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from itertools import combinations

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import scale
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler


Load the data.

In [2]:
df = pd.read_csv("ames.csv")

In [3]:
df = df[['LotArea', 'OverallQual', 'OverallCond', 'TotalBsmtSF',
         '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'TotRmsAbvGrd',
         'GarageArea', 'Fireplaces', 'SalePrice']]

## Look at a baseline housing data model

Above, we imported the Ames housing data and grabbed a subset of the data to use in this analysis.

Next steps:

- Split the data into target (`y`) and predictors (`X`) -- ensure these both are DataFrames 
- Scale all the predictors using `scale`. Convert these scaled features into a DataFrame 
- Build at a baseline model using *scaled variables* as predictors. Use 5-fold cross-validation (set `random_state` to 1) and use the $R^2$ score to evaluate the model 

In [4]:
# Your code here
X = df.drop('SalePrice', axis = 1)
y = df['SalePrice']
ss = StandardScaler()
X_scaled = ss.fit_transform(X)

In [5]:
lr = LinearRegression()
kf = KFold(n_splits = 5, random_state = 1, shuffle = True)
cross_val_score(lr, X_scaled, y, cv = kf).mean()

0.7524751004088887

## Include interactions

Look at all the possible combinations of variables for interactions by adding interactions one by one to the baseline model. Next, evaluate that model using 5-fold cross-validation and store the $R^2$ to compare it with the baseline model.

Print the 7 most important interactions.

In [6]:
# Your code here
X_scaled = pd.DataFrame(X_scaled, columns = X.columns)
X_scaled.head()

LotArea  OverallQual  OverallCond  TotalBsmtSF  1stFlrSF  2ndFlrSF  \
0 -0.207142     0.651479    -0.517200    -0.459303 -0.793434  1.161852   
1 -0.091886    -0.071836     2.179628     0.466465  0.257140 -0.795163   
2  0.073480     0.651479    -0.517200    -0.313369 -0.627826  1.189351   
3 -0.096897     0.651479    -0.517200    -0.687324 -0.521734  0.937276   
4  0.375148     1.374795    -0.517200     0.199680 -0.045611  1.617877   

   GrLivArea  TotRmsAbvGrd  GarageArea  Fireplaces  
0   0.370333      0.912210    0.351000   -0.951226  
1  -0.482512     -0.318683   -0.060731    0.600495  
2   0.515013     -0.318683    0.631726    0.600495  
3   0.383659      0.296763    0.790804    0.600495  
4   1.299326      1.527656    1.698485    0.600495

In [7]:
garbage = []
for x in list(combinations(X_scaled.columns, 2)):
    var1 = x[0]
    var2 = x[1]
    X_scaled['new_feat'] = X_scaled[var1] * X_scaled[var2]
    lr = LinearRegression()
    kf = KFold(n_splits = 5, random_state = 1, shuffle = True)
    junk = cross_val_score(lr, X_scaled, y, cv = kf).mean()
    my_dict = {'var1': var1, 'var2': var2, 'R2': junk}
    garbage.append(my_dict)

In [8]:
garbage_df = pd.DataFrame(garbage)
garbage_df.sort_values(by = 'R2', ascending = False).head(7)

var1          var2        R2
14  OverallQual  TotRmsAbvGrd  0.769742
15  OverallQual    GarageArea  0.763705
12  OverallQual      2ndFlrSF  0.757779
36     2ndFlrSF  TotRmsAbvGrd  0.756143
35     2ndFlrSF     GrLivArea  0.756001
16  OverallQual    Fireplaces  0.754232
17  OverallCond   TotalBsmtSF  0.754090

Write code to include the 7 most important interactions in your data set by adding 7 columns. Name the columns "var1_var2", where var1 and var2 are the two variables in the interaction.

In [9]:
X_scaled.drop('new_feat', axis = 1, inplace = True)

In [10]:
# Your code here
for index,row in garbage_df.sort_values(by = 'R2', ascending = False).head(7).iterrows():
    X_scaled[f'{row["var1"]}_{row["var2"]}'] = X_scaled[row['var1']] * X_scaled[row['var2']]

In [11]:
X_scaled.head()

LotArea  OverallQual  OverallCond  TotalBsmtSF  1stFlrSF  2ndFlrSF  \
0 -0.207142     0.651479    -0.517200    -0.459303 -0.793434  1.161852   
1 -0.091886    -0.071836     2.179628     0.466465  0.257140 -0.795163   
2  0.073480     0.651479    -0.517200    -0.313369 -0.627826  1.189351   
3 -0.096897     0.651479    -0.517200    -0.687324 -0.521734  0.937276   
4  0.375148     1.374795    -0.517200     0.199680 -0.045611  1.617877   

   GrLivArea  TotRmsAbvGrd  GarageArea  Fireplaces  OverallQual_TotRmsAbvGrd  \
0   0.370333      0.912210    0.351000   -0.951226                  0.594286   
1  -0.482512     -0.318683   -0.060731    0.600495                  0.022893   
2   0.515013     -0.318683    0.631726    0.600495                 -0.207616   
3   0.383659      0.296763    0.790804    0.600495                  0.193335   
4   1.299326      1.527656    1.698485    0.600495                  2.100214   

   OverallQual_GarageArea  OverallQual_2ndFlrSF  2ndFlrSF_TotRmsAbvGrd  \
0                0.228669              0.756922               1.059852   
1                0.004363              0.057121               0.253405   
2                0.411557              0.774837              -0.379026   
3                0.515193              0.610616               0.278149   
4                2.335068              2.224249               2.471560   

   2ndFlrSF_GrLivArea  OverallQual_Fireplaces  OverallCond_TotalBsmtSF  
0            0.430272               -0.619704                 0.237551  
1            0.383676               -0.043137                 1.016720  
2            0.612531                0.391210                 0.162074  
3            0.359595                0.391210                 0.355484  
4            2.102150                0.825557                -0.103274

## Include polynomials

Try polynomials of degrees 2, 3, and 4 for each variable, in a similar way you did for interactions (by looking at your baseline model and seeing how $R^2$ increases). Do understand that when going for a polynomial of 4, the particular column is raised to the power of 2 and 3 as well in other terms. We only want to include "pure" polynomials, so make sure no interactions are included. We want the result to return a list that contain tuples of the form:

`(var_name, degree, R2)`, so eg. `('OverallQual', 2, 0.781)` 

In [12]:
# Your code here
garbage = []
for warlock in X.columns:
    X_scaled_copy = X_scaled.copy()
    for michaelbolton in range(2, 5):
        X_scaled_copy[f'{warlock}_{michaelbolton}'] = X_scaled_copy[warlock]**michaelbolton
        lr = LinearRegression()
        kf = KFold(n_splits = 5, random_state = 1, shuffle = True)
        junk = cross_val_score(lr, X_scaled_copy, y, cv = kf).mean()
        my_dict = {'var_name': warlock, 'degree': michaelbolton, 'R2': junk}
        garbage.append(my_dict)

In [13]:
garbage_poly_df = pd.DataFrame(garbage)
garbage_poly_df.head()

var_name  degree        R2
0      LotArea       2  0.760283
1      LotArea       3  0.576393
2      LotArea       4 -1.549865
3  OverallQual       2  0.767559
4  OverallQual       3  0.767063

For each variable, print out the maximum $R^2$ possible when including Polynomials.

In [14]:
# Your code here
marissa_dict = {'R2': 'max', 'degree': 'first'}
garbage_poly_df.sort_values('R2', ascending=False).groupby(['var_name']).agg(marissa_dict).reset_index().sort_values('R2', 
                                                                                                                    ascending=False)

var_name        R2  degree
4     GrLivArea  0.832458       4
0      1stFlrSF  0.795879       2
1      2ndFlrSF  0.790912       3
9   TotalBsmtSF  0.776991       2
7   OverallQual  0.767559       2
3    GarageArea  0.762471       2
6   OverallCond  0.762412       2
2    Fireplaces  0.761644       2
8  TotRmsAbvGrd  0.760291       2
5       LotArea  0.760283       2

Which two variables seem to benefit most from adding polynomial terms?

Add Polynomials for the two features that seem to benefit the most, as in have the best $R^2$ compared to the baseline model. For each of the two features, raise to the Polynomial that generates the best result. Make sure to start from the data set `df_inter` so the final data set has both interactions and polynomials in the model.

In [15]:
# Your code here
X_scaled['GrLivArea_2'] = X_scaled['GrLivArea']**2
X_scaled['GrLivArea_3'] = X_scaled['GrLivArea']**3
X_scaled['GrLivArea_4'] = X_scaled['GrLivArea']**4
X_scaled['1stFlrSF_2'] = X_scaled['1stFlrSF']**2

Check out your final data set and make sure that your interaction terms as well as your polynomial terms are included.

In [16]:
# Your code here
X_scaled.head()

LotArea  OverallQual  OverallCond  TotalBsmtSF  1stFlrSF  2ndFlrSF  \
0 -0.207142     0.651479    -0.517200    -0.459303 -0.793434  1.161852   
1 -0.091886    -0.071836     2.179628     0.466465  0.257140 -0.795163   
2  0.073480     0.651479    -0.517200    -0.313369 -0.627826  1.189351   
3 -0.096897     0.651479    -0.517200    -0.687324 -0.521734  0.937276   
4  0.375148     1.374795    -0.517200     0.199680 -0.045611  1.617877   

   GrLivArea  TotRmsAbvGrd  GarageArea  Fireplaces  ...  \
0   0.370333      0.912210    0.351000   -0.951226  ...   
1  -0.482512     -0.318683   -0.060731    0.600495  ...   
2   0.515013     -0.318683    0.631726    0.600495  ...   
3   0.383659      0.296763    0.790804    0.600495  ...   
4   1.299326      1.527656    1.698485    0.600495  ...   

   OverallQual_GarageArea  OverallQual_2ndFlrSF  2ndFlrSF_TotRmsAbvGrd  \
0                0.228669              0.756922               1.059852   
1                0.004363              0.057121               0.253405   
2                0.411557              0.774837              -0.379026   
3                0.515193              0.610616               0.278149   
4                2.335068              2.224249               2.471560   

   2ndFlrSF_GrLivArea  OverallQual_Fireplaces  OverallCond_TotalBsmtSF  \
0            0.430272               -0.619704                 0.237551   
1            0.383676               -0.043137                 1.016720   
2            0.612531                0.391210                 0.162074   
3            0.359595                0.391210                 0.355484   
4            2.102150                0.825557                -0.103274   

   GrLivArea_2  GrLivArea_3  GrLivArea_4  1stFlrSF_2  
0     0.137147     0.050790     0.018809    0.629537  
1     0.232818    -0.112337     0.054204    0.066121  
2     0.265238     0.136601     0.070351    0.394166  
3     0.147194     0.056472     0.021666    0.272206  
4     1.688247     2.193583     2.850179    0.002080  

[5 rows x 21 columns]

## Full model R-squared

Check out the $R^2$ of the full model.

In [18]:
# Your code here
lr = LinearRegression()
kf = KFold(n_splits = 5, random_state = 1, shuffle = True)
junk = cross_val_score(lr, X_scaled, y, cv = kf).mean()
print(junk)

0.834448037522759


## Find the best Lasso regularization parameter

You learned that when using Lasso regularization, your coefficients shrink to 0 when using a higher regularization parameter. Now the question is which value we should choose for the regularization parameter. 

This is where the AIC and BIC come in handy! We'll use both criteria in what follows and perform cross-validation to select an optimal value of the regularization parameter $alpha$ of the Lasso estimator.

Read the page here: https://scikit-learn.org/stable/auto_examples/linear_model/plot_lasso_model_selection.html and create a similar plot as the first one listed on the page. 

In [20]:
from sklearn.linear_model import Lasso, LassoCV, LassoLarsCV, LassoLarsIC

In [21]:
# Your code here 
AIC_garbage = LassoLarsIC(criterion="aic", normalize=False).fit(X_scaled, y)

In [30]:
fig, ax = plt.subplots()
plt.plot(AIC_garbage.alphas_, AIC_garbage.criterion_)
plt.axvline(AIC_garbage.alpha_)
ax.set_xscale("log")

In [31]:
BIC_garbage = LassoLarsIC(criterion="bic", normalize=False).fit(X_scaled, y)
fig, ax = plt.subplots()
plt.plot(BIC_garbage.alphas_, BIC_garbage.criterion_)
plt.axvline(BIC_garbage.alpha_)
ax.set_xscale("log")

## Analyze the final result

Finally, use the best value for the regularization parameter according to AIC and BIC, and compare $R^2$ and RMSE using train-test split. Compare with the baseline model.

Remember, you can find the Root Mean Squared Error (RMSE) by setting `squared=False` inside the function (see [the documentation](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html)), and the RMSE returns values that are in the same units as our target - so we can see how far off our predicted sale prices are in dollars.

In [32]:
from sklearn.metrics import mean_squared_error, mean_squared_log_error
from sklearn.model_selection import train_test_split

In [35]:
# Split X_scaled and y into training and test sets
# Set random_state to 1
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y)

# Code for baseline model
linreg_all = LinearRegression()
linreg_all.fit(X_train, y_train)


# Print R-Squared and RMSE
print(linreg_all.score(X_test, y_test))
print(mean_squared_error(y_test, linreg_all.predict(X_test), squared=False))

0.7892134737957821
35109.93854172202


In [36]:
# Split df_inter and y into training and test sets
# Set random_state to 1
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y)

# Code for lasso with alpha from AIC
lasso = Lasso(alpha=AIC_garbage.alpha_)
lasso.fit(X_train, y_train)


# Print R-Squared and RMSE
print(lasso.score(X_test, y_test))
print(mean_squared_error(y_test, lasso.predict(X_test), squared=False))

0.8463411422024139
34465.74714519847


In [37]:
# Code for lasso with alpha from BIC
lasso = Lasso(alpha=BIC_garbage.alpha_)
lasso.fit(X_train, y_train)


# Print R-Squared and RMSE
print(lasso.score(X_test, y_test))
print(mean_squared_error(y_test, lasso.predict(X_test), squared=False))

0.8450038453338913
34615.40036978014


## Level up (Optional)

### Create a Lasso path

From this section, you know that when using Lasso, more parameters shrink to zero as your regularization parameter goes up. In Scikit-learn there is a function `lasso_path()` which visualizes the shrinkage of the coefficients while $alpha$ changes. Try this out yourself!

https://scikit-learn.org/stable/auto_examples/linear_model/plot_lasso_coordinate_descent_path.html#sphx-glr-auto-examples-linear-model-plot-lasso-coordinate-descent-path-py

### AIC and BIC for subset selection
This notebook shows how you can use AIC and BIC purely for feature selection. Try this code out on our Ames housing data!

https://xavierbourretsicotte.github.io/subset_selection.html

## Summary

Congratulations! You now know how to create better linear models and how to use AIC and BIC for both feature selection and to optimize your regularization parameter when performing Ridge and Lasso. 